In [1]:
from torchvision import transforms, models
# from sklearn.manifold import TSNE
from PIL import Image
from torch import nn
import pandas as pd
import numpy as np
import argparse
import random
import torch
import time
import copy
import os

from dataloader import *
from utils import *

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [2]:
records_dir = 'test'
os.mkdir(records_dir)
os.mkdir(os.path.join(records_dir, "models"))  

In [3]:
t = pd.read_csv("../../clean_data/wealth_random_points_cumsum_jenks.csv")
t = t[t["iso2"] == "HT"]
t.head()

,DHSID,DHSYEAR,DHSCLUST,survey,folder,buffer_siz,hv001,hv271,geometry,points,random_points,val,cumsum,longitude,latitude,iso2,class,jenks_class,wealth_class
24120,HT200600000001,2006.0,1.0,222,HTGE52FL,5,1,11588.076923,"POLYGON ((-74.2636224219644 18.58489728485856,...",10,"(-74.27848710579512, 18.536657860403118)",1,1,-74.278487,18.536658,HT,"(-295754.146, 212834.062]",2,2
24121,HT200600000001,2006.0,1.0,222,HTGE52FL,5,1,11588.076923,"POLYGON ((-74.2636224219644 18.58489728485856,...",10,"(-74.27025761210759, 18.545985153645116)",1,2,-74.270258,18.545985,HT,"(-295754.146, 212834.062]",2,2
24122,HT200600000001,2006.0,1.0,222,HTGE52FL,5,1,11588.076923,"POLYGON ((-74.2636224219644 18.58489728485856,...",10,"(-74.30264405930176, 18.559220088841535)",1,3,-74.302644,18.559220,HT,"(-295754.146, 212834.062]",2,2
24123,HT200600000001,2006.0,1.0,222,HTGE52FL,5,1,11588.076923,"POLYGON ((-74.2636224219644 18.58489728485856,...",10,"(-74.28281896593455, 18.576832720091083)",1,4,-74.282819,18.576833,HT,"(-295754.146, 212834.062]",2,2
24124,HT200600000001,2006.0,1.0,222,HTGE52FL,5,1,11588.076923,"POLYGON ((-74.2636224219644 18.58489728485856,...",10,"(-74.28183772482326, 18.578955152011318)",1,5,-74.281838,18.578955,HT,"(-295754.146, 212834.062]",2,2


In [4]:
t["wealth_class"].value_counts()

1    1800
3     790
2     540
0     140
Name: wealth_class, dtype: int64

In [5]:
ecup = f"../../imagery/HT/"

In [11]:
image_names = [ecup + _ for _ in os.listdir(ecup)]

# with open(f"./ht_v2/valimages.txt", "r") as f:
#     image_names = f.read().splitlines()
# image_names = [i for i in image_names if ".ipynb" not in i]

data = Dataloader(image_names, "../../clean_data/wealth_random_points_cumsum_jenks.csv", records_dir, batch_size = 32)

In [12]:
device = "cuda"
model_ft = models.resnet50(pretrained = True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 4)
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=0.0001)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)    

In [15]:
# inp, out = data.load_data(1)
# inp = inp.to(device)
# # inp

In [16]:
# torch.max(w(inp), 1)

In [17]:
"""
torch.return_types.max(
values=tensor([1.1792, 2.1368, 1.0662, 1.2713, 1.8283, 0.8798, 1.6577, 1.2981, 1.2897,
        3.3371, 1.8538, 1.2088, 1.5757, 1.4533, 0.6662, 3.4274, 1.7795, 1.5605,
        0.6932, 0.9968, 1.0821, 1.0260, 1.7533, 0.6047, 0.9628, 1.4067, 0.7750,
        1.8818, 3.5705, 1.3958, 2.2733, 1.1322], device='cuda:0',
       grad_fn=<MaxBackward0>),
indices=tensor([1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1,
        2, 1, 1, 1, 3, 1, 1, 1], device='cuda:0'))
"""

"\ntorch.return_types.max(\nvalues=tensor([1.1792, 2.1368, 1.0662, 1.2713, 1.8283, 0.8798, 1.6577, 1.2981, 1.2897,\n        3.3371, 1.8538, 1.2088, 1.5757, 1.4533, 0.6662, 3.4274, 1.7795, 1.5605,\n        0.6932, 0.9968, 1.0821, 1.0260, 1.7533, 0.6047, 0.9628, 1.4067, 0.7750,\n        1.8818, 3.5705, 1.3958, 2.2733, 1.1322], device='cuda:0',\n       grad_fn=<MaxBackward0>),\nindices=tensor([1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1,\n        2, 1, 1, 1, 3, 1, 1, 1], device='cuda:0'))\n"

In [18]:
# torch.save({
# #         'epoch': epoch,
#         'model_state_dict': w.state_dict(),
# #         'optimizer_state_dict': optimizer.state_dict(),
#         'loss': criterion,
#     }, f"./test/model_epoch1.torch")

In [19]:
import joblib


model_ft = joblib.load("./ht_v4/models/model_epoch0.sav")
model_ft.eval();

In [20]:
# testm = torch.load("./ht_v2/models/model_epoch0.torch")["model_state_dict"]
# model_ft.load_state_dict(testm)
# model_ft.eval()

In [22]:
# inp.shape

In [31]:
model_ft.conv1.weight.shape

torch.Size([64, 3, 7, 7])

In [27]:
for i in range(0, 10):
    inp, out = data.load_data(i, "train")
    inp, out = inp.to(device), out.to(device)
    preds = torch.max(model_ft(inp), 1)[1]
    print(preds, out)
    print(torch.sum(preds == out.data), "\n\n")

tensor([1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1,
        3, 1, 1, 1, 3, 1, 1, 1], device='cuda:0') tensor([2, 1, 1, 3, 1, 3, 2, 0, 2, 3, 3, 2, 1, 2, 1, 3, 1, 1, 2, 1, 3, 1, 1, 1,
        3, 1, 1, 1, 1, 1, 3, 1], device='cuda:0')
tensor(20, device='cuda:0') 


tensor([3, 1, 1, 3, 3, 3, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0') tensor([3, 2, 1, 3, 3, 3, 2, 1, 1, 3, 1, 1, 2, 3, 3, 1, 1, 1, 1, 1, 2, 1, 1, 0,
        2, 1, 1, 3, 1, 2, 3, 1], device='cuda:0')
tensor(20, device='cuda:0') 


tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0') tensor([1, 2, 2, 1, 0, 1, 3, 2, 2, 1, 3, 3, 2, 3, 2, 2, 3, 1, 1, 3, 1, 1, 2, 1,
        0, 2, 3, 2, 2, 0, 3, 1], device='cuda:0')
tensor(11, device='cuda:0') 


tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 1,
        1, 1, 1, 1, 1, 1, 3, 1], device='cuda:0